## Switch to CPU Instance (Advisable only for Non Colab-Pro instance)

1. Switch to CPU Instance for until Step 2 for non GPU dependent tasks
2. This increases your time available for the GPU dependent tasks on a Colab instance
2. Change Runtime type to CPU by Runtime(Top Left tab)->Change Runtime Type->None(Hardware Accelerator)
3.   Then click on Connect (Top Right)



## Mounting Google drive
Mount your Google drive storage to this Colab instance

In [ ]:
try:
    import google.colab
    %env GOOGLE_COLAB=1
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except:
    %env GOOGLE_COLAB=0
    print("Warning: Not a Colab Environment")

# Object Detection using TAO YOLOv3

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080">


## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained ResNet-18 model and train a ResNet-18 YOLO v3 model on the KITTI dataset
* Prune the trained YOLO v3 model
* Retrain the pruned model to recover lost accuracy
* Export the pruned model
* Quantize the pruned model using QAT
* Run Inference on the trained model
* Export the pruned, quantized and retrained model to a .etlt file for deployment to DeepStream

## Table of Contents

This notebook shows an example usecase of YOLO v3 object detection using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables](#head-0)
1. [Prepare dataset and pre-trained model](#head-1) <br>
     1.1 [Download the dataset](#head-1-1)<br>
     1.2 [Verify the downloaded dataset](#head-1-2)<br>
     1.3 [Download pretrained model](#head-1-3)
2. [Setup GPU environment](#head-2) <br>
    2.1 [Connect to GPU Instance](#head-2-1) <br>
    2.2 [Mounting Google drive](#head-2-2) <br>
    2.3 [Setup Python environment](#head-2-3) <br>
    2.4 [Reset env variables](#head-2-4) <br>
3. [Generate TF Records](#head-3)
4. [Provide training specification](#head-4)
5. [Run TAO training](#head-5)
6. [Evaluate trained models](#head-6)
7. [Prune trained models](#head-7)
8. [Retrain pruned models](#head-8)
9. [Evaluate retrained model](#head-9)
10. [Visualize inferences](#head-10)

## 0. Set up env variables<a class="anchor" id="head-0"></a>

When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

*Note: Please make sure to remove any stray artifacts/files from the `$EXPERIMENT_DIR` or `$DATA_DIR` paths as mentioned below, that may have been generated from previous experiments. Having checkpoint files etc may interfere with creating a training graph for a new experiment.*

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%env TAO_DOCKER_DISABLE=1

%env KEY=nvidia_tlt
%env NUM_GPUS=1

# Change the paths according to your directory structure, these are just examples
%env COLAB_NOTEBOOKS_PATH=/content/drive/MyDrive/ColabNotebooks
if not os.path.exists(os.environ["COLAB_NOTEBOOKS_PATH"]):
    raise("Error, enter the path of the colab notebooks repo correctly")
%env EXPERIMENT_DIR=/results/yolo_v3
%env DATA_DIR=/content/drive/MyDrive/kitti_data/

SPECS_DIR=f"{os.environ['COLAB_NOTEBOOKS_PATH']}/tensorflow/yolo_v3/specs"
%env SPECS_DIR={SPECS_DIR}
# Showing list of specification files.
!ls -rlt $SPECS_DIR

!sudo mkdir -p $DATA_DIR && sudo chmod -R 777 $DATA_DIR
!sudo mkdir -p $EXPERIMENT_DIR && sudo chmod -R 777 $EXPERIMENT_DIR

## 1. Prepare dataset and pre-trained model <a class="anchor" id="head-1"></a>

 We will be using the KITTI detection dataset for the tutorial. To find more details please visit
 http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d. Please download the KITTI detection images (http://www.cvlibs.net/download.php?file=data_object_image_2.zip) and labels (http://www.cvlibs.net/download.php?file=data_object_label_2.zip) to $DATA_DIR.
 
 The data will then be extracted to have
 * training images in `$DATA_DIR/training/image_2`
 * training labels in `$DATA_DIR/training/label_2`
 * testing images in `$DATA_DIR/testing/image_2`
 
You may use this notebook with your own dataset as well. To use this example with your own dataset, please follow the same directory structure as mentioned below.

*Note: There are no labels for the testing images, therefore we use it just to visualize inferences for the trained model.*

### 1.1 Download the dataset <a class="anchor" id="head-1-1"></a>

Once you have gotten the download links in your email, please populate them in place of the `KITTI_IMAGES_DOWNLOAD_URL` and the `KITTI_LABELS_DOWNLOAD_URL`. This next cell, will download the data and place in `$DATA_DIR`

In [ ]:
import os
!mkdir -p $DATA_DIR
os.environ["URL_IMAGES"]=KITTI_IMAGES_DOWNLOAD_URL
!if [ ! -f $DATA_DIR/data_object_image_2.zip ]; then wget $URL_IMAGES -O $DATA_DIR/data_object_image_2.zip; else echo "image archive already downloaded"; fi 
os.environ["URL_LABELS"]=KITTI_LABELS_DOWNLOAD_URL
!if [ ! -f $DATA_DIR/data_object_label_2.zip ]; then wget $URL_LABELS -O $DATA_DIR/data_object_label_2.zip; else echo "label archive already downloaded"; fi 

### 1.2 Verify the downloaded dataset <a class="anchor" id="head-1-2"></a>

In [ ]:
# Check the dataset is present
!mkdir -p $DATA_DIR
!if [ ! -f $DATA_DIR/data_object_image_2.zip ]; then echo 'Image zip file not found, please download.'; else echo 'Found Image zip file.';fi
!if [ ! -f $DATA_DIR/data_object_label_2.zip ]; then echo 'Label zip file not found, please download.'; else echo 'Found Labels zip file.';fi

In [ ]:
# This may take a while: verify integrity of zip files 
!sha256sum $DATA_DIR/data_object_image_2.zip | cut -d ' ' -f 1 | grep -xq '^351c5a2aa0cd9238b50174a3a62b846bc5855da256b82a196431d60ff8d43617$' ; \
if test $? -eq 0; then echo "images OK"; else echo "images corrupt, re-download!" && rm -f $DATA_DIR/data_object_image_2.zip; fi 
!sha256sum $DATA_DIR/data_object_label_2.zip | cut -d ' ' -f 1 | grep -xq '^4efc76220d867e1c31bb980bbf8cbc02599f02a9cb4350effa98dbb04aaed880$' ; \
if test $? -eq 0; then echo "labels OK"; else echo "labels corrupt, re-download!" && rm -f $DATA_DIR/data_object_label_2.zip; fi 

In [ ]:
# unpack 
!unzip -u $DATA_DIR/data_object_image_2.zip -d $DATA_DIR
!unzip -u $DATA_DIR/data_object_label_2.zip -d $DATA_DIR

In [ ]:
# verify
import os

DATA_DIR = os.environ.get('DATA_DIR')
num_training_images = len(os.listdir(os.path.join(DATA_DIR, "training/image_2")))
num_training_labels = len(os.listdir(os.path.join(DATA_DIR, "training/label_2")))
num_testing_images = len(os.listdir(os.path.join(DATA_DIR, "testing/image_2")))
print("Number of images in the train/val set. {}".format(num_training_images))
print("Number of labels in the train/val set. {}".format(num_training_labels))
print("Number of images in the test set. {}".format(num_testing_images))

In [ ]:
# Sample kitti label.
!cat $DATA_DIR/training/label_2/000110.txt

### 1.3 Download pre-trained model <a class="anchor" id="head-1-3"></a>

We will use NGC CLI to get the pre-trained models. For more details, go to [ngc.nvidia.com](ngc.nvidia.com) and click the SETUP on the navigation bar.

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
%env LOCAL_PROJECT_DIR=/ngc_content/
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u -q "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))
!cp /usr/lib/x86_64-linux-gnu/libstdc++.so.6 $LOCAL_PROJECT_DIR/ngccli/ngc-cli/libstdc++.so.6

In [ ]:
!ngc registry model list nvidia/tao/pretrained_object_detection:*

In [ ]:
!mkdir -p $EXPERIMENT_DIR/pretrained_resnet18/

In [ ]:
# Pull pretrained model from NGC
!ngc registry model download-version nvidia/tao/pretrained_object_detection:resnet18 \
                    --dest $EXPERIMENT_DIR/pretrained_resnet18

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $EXPERIMENT_DIR/pretrained_resnet18/pretrained_object_detection_vresnet18

## 2. Setup GPU environment <a class="anchor" id="head-2"></a>


### 2.1 Connect to GPU Instance <a class="anchor" id="head-2-1"></a>

1. Move any data saved to the Colab Instance storage to Google Drive  
2. Change Runtime type to GPU by Runtime(Top Left tab)->Change Runtime Type->GPU(Hardware Accelerator)
3.   Then click on Connect (Top Right)



### 2.2 Mounting Google drive <a class="anchor" id="head-2-2"></a>
Mount your Google drive storage to this Colab instance

In [ ]:
try:
    import google.colab
    %env GOOGLE_COLAB=1
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except:
    %env GOOGLE_COLAB=0
    print("Warning: Not a Colab Environment")

### 2.3 Setup Python environment <a class="anchor" id="head-2-3"></a>
Setup the environment necessary to run the TAO Networks by running the bash script

In [ ]:
#FIXME
%env GENERAL_WHL_PATH=/content/drive/MyDrive/tf/general_whl
#FIXME
%env CODEBASE_WHL_PATH=/content/drive/MyDrive/tf/codebase_whl

import os
if os.path.exists(os.environ["GENERAL_WHL_PATH"]) and os.path.exists(os.environ["GENERAL_WHL_PATH"]):
    if os.environ["GOOGLE_COLAB"] == "1":
        os.environ["bash_script"] = "setup_env.sh"
    else:
        os.environ["bash_script"] = "setup_env_desktop.sh"

    !sed -i "s|PATH_TO_GENERAL_WHL|$GENERAL_WHL_PATH|g" $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script
    !sed -i "s|PATH_TO_CODEBASE_WHL|$CODEBASE_WHL_PATH|g" $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script
    !sed -i "s|PATH_TO_COLAB_NOTEBOOKS|$COLAB_NOTEBOOKS_PATH|g" $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script

    !sh $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script
else:
    raise("Error, enter the whl paths correctly")

In [ ]:
if os.environ.get("PYTHONPATH","") == "":
    os.environ["PYTHONPATH"] = ""
os.environ["PYTHONPATH"]+=":/opt/nvidia/"
if os.environ["GOOGLE_COLAB"] == "1":
    os.environ["PYTHONPATH"]+=":/usr/local/lib/python3.6/dist-packages/third_party/nvml"
else:
    os.environ["PYTHONPATH"]+=":/home_duplicate/rarunachalam/miniconda3/envs/tf_py_36/lib/python3.6/site-packages/third_party/nvml" # FIX MINICONDA PATH

### 2.4 Reset env variables <a class="anchor" id="head-2-4"></a>

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%env TAO_DOCKER_DISABLE=1

%env KEY=nvidia_tlt
%env NUM_GPUS=1

# Change the paths according to your directory structure, these are just examples
%env COLAB_NOTEBOOKS_PATH=/content/drive/MyDrive/ColabNotebooks
if not os.path.exists(os.environ["COLAB_NOTEBOOKS_PATH"]):
    raise("Error, enter the path of the colab notebooks repo correctly")
%env EXPERIMENT_DIR=/results/yolo_v3
%env DATA_DIR=/content/drive/MyDrive/kitti_data/

SPECS_DIR=f"{os.environ['COLAB_NOTEBOOKS_PATH']}/tensorflow/yolo_v3/specs"
%env SPECS_DIR={SPECS_DIR}
# Showing list of specification files.
!ls -rlt $SPECS_DIR

## 3. Generate TF records <a class="anchor" id="head-3"></a>

Either TFRecord dataset format or the raw KITTI format(with directories of images and KITTI labels) can be used for training/validation. TFRecord dataset format is the recommended one as it is generally faster than reading numerious KITTI label files. However, in some cases, especially when training with small resolutions like 416x416, the experimental result shows TFRecord dataset is almost the same or slightly slower than the other format. So by default, we will use TFRecord dataset in this notebook. As a reference, we still provide the spec files for raw KITTI dataset format.

In [ ]:
!sed -i "s|TAO_DATA_PATH|$DATA_DIR/|g" $SPECS_DIR/tfrecords_kitti_trainval.txt
!sed -i "s|EXPERIMENT_DIR_PATH|$EXPERIMENT_DIR/|g" $SPECS_DIR/tfrecords_kitti_trainval.txt

print("TFrecords conversion spec file for training")
!cat $SPECS_DIR/tfrecords_kitti_trainval.txt

In [ ]:
# KITTI trainval
!tao yolo_v3 dataset_convert -d $SPECS_DIR/tfrecords_kitti_trainval.txt \
                     -o $DATA_DIR/tfrecords/kitti_trainval/kitti_trainval

In [ ]:
# If you use your own dataset, you will need to run the code below to generate the best anchor shape

# !tao yolo_v3 kmeans -l $DATA_DIR/training/label_2 \
#                     -i $DATA_DIR/training/image_2 \
#                     -n 9 \
#                     -x 1248 \
#                     -y 384

# The anchor shape generated by this script is sorted. Write the first 3 into small_anchor_shape in the config
# file. Write middle 3 into mid_anchor_shape. Write last 3 into big_anchor_shape.

## 4. Provide training specification <a class="anchor" id="head-4"></a>
* Augmentation parameters for on-the-fly data augmentation
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate etc.
* Whether to use quantization aware training (QAT)

In [ ]:
# Provide pretrained model path
!sed -i "s|TAO_DATA_PATH|$DATA_DIR/|g" $SPECS_DIR/yolo_v3_train_resnet18_tfrecord.txt
!sed -i "s|EXPERIMENT_DIR_PATH|$EXPERIMENT_DIR/|g" $SPECS_DIR/yolo_v3_train_resnet18_tfrecord.txt

# To enable QAT training on sample spec file, uncomment following lines
# !sed -i "s/enable_qat: false/enable_qat: true/g" $SPECS_DIR/yolo_v3_train_resnet18_tfrecord.txt
# !sed -i "s/enable_qat: false/enable_qat: true/g" $SPECS_DIR/yolo_v3_retrain_resnet18_tfrecord.txt

In [ ]:
# By default, the sample spec file disables QAT training. You can force non-QAT training by running lines below
# !sed -i "s/enable_qat: true/enable_qat: false/g" $SPECS_DIR/yolo_v3_train_resnet18_tfrecord.txt
# !sed -i "s/enable_qat: true/enable_qat: false/g" $SPECS_DIR/yolo_v3_retrain_resnet18_tfrecord.txt

In [ ]:
!cat $SPECS_DIR/yolo_v3_train_resnet18_tfrecord.txt

## 5. Run TAO training <a class="anchor" id="head-5"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

In [ ]:
!mkdir -p $EXPERIMENT_DIR/experiment_dir_unpruned

In [ ]:
print("To run with multigpu, please change --gpus based on the number of available GPUs in your machine.")
!tao yolo_v3 train -e $SPECS_DIR/yolo_v3_train_resnet18_tfrecord.txt \
                   -r $EXPERIMENT_DIR/experiment_dir_unpruned \
                   -k $KEY \
                   --gpus 1

In [ ]:
print("To resume from checkpoint, please change pretrain_model_path to resume_model_path in config file.")

In [ ]:
print('Model for each epoch:')
print('---------------------')
!ls -ltrh $EXPERIMENT_DIR/experiment_dir_unpruned/weights

In [ ]:
# Now check the evaluation stats in the csv file and pick the model with highest eval accuracy.
!cat $EXPERIMENT_DIR/experiment_dir_unpruned/yolov3_training_log_resnet18.csv
%env EPOCH=080

## 6. Evaluate trained models <a class="anchor" id="head-6"></a>

In [ ]:
!tao yolo_v3 evaluate -e $SPECS_DIR/yolo_v3_train_resnet18_tfrecord.txt \
                      -m $EXPERIMENT_DIR/experiment_dir_unpruned/weights/yolov3_resnet18_epoch_$EPOCH.tlt \
                      -k $KEY

## 7. Prune trained models <a class="anchor" id="head-7"></a>
* Specify pre-trained model
* Equalization criterion (`Only for resnets as they have element wise operations or MobileNets.`)
* Threshold for pruning.
* A key to save and load the model
* Output directory to store the model

Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold value depends on the dataset and the model. `0.5` in the block below is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.

In [ ]:
!mkdir -p $EXPERIMENT_DIR/experiment_dir_pruned

In [ ]:
!tao yolo_v3 prune -m $EXPERIMENT_DIR/experiment_dir_unpruned/weights/yolov3_resnet18_epoch_$EPOCH.tlt \
                   -e $SPECS_DIR/yolo_v3_train_resnet18_tfrecord.txt \
                   -o $EXPERIMENT_DIR/experiment_dir_pruned/yolov3_resnet18_pruned.tlt \
                   -eq intersection \
                   -pth 0.1 \
                   -k $KEY

In [ ]:
!ls -rlt $EXPERIMENT_DIR/experiment_dir_pruned/

## 8. Retrain pruned models <a class="anchor" id="head-8"></a>
* Model needs to be re-trained to bring back accuracy after pruning
* Specify re-training specification
* WARNING: training will take several hours or one day to complete

In [ ]:
# Printing the retrain spec file. 
# Here we have updated the spec file to include the newly pruned model as a pretrained weights.
!sed -i "s|TAO_DATA_PATH|$DATA_DIR/|g" $SPECS_DIR/yolo_v3_retrain_resnet18_tfrecord.txt
!sed -i "s|EXPERIMENT_DIR_PATH|$EXPERIMENT_DIR/|g" $SPECS_DIR/yolo_v3_retrain_resnet18_tfrecord.txt
!cat $SPECS_DIR/yolo_v3_retrain_resnet18_tfrecord.txt

In [ ]:
!mkdir -p $EXPERIMENT_DIR/experiment_dir_retrain

In [ ]:
# Retraining using the pruned model as pretrained weights 
!tao yolo_v3 train --gpus 1 \
                   -e $SPECS_DIR/yolo_v3_retrain_resnet18_tfrecord.txt \
                   -r $EXPERIMENT_DIR/experiment_dir_retrain \
                   -k $KEY

In [ ]:
# Listing the newly retrained model.
!ls -rlt $EXPERIMENT_DIR/experiment_dir_retrain/weights

In [ ]:
# Now check the evaluation stats in the csv file and pick the model with highest eval accuracy.
!cat $EXPERIMENT_DIR/experiment_dir_retrain/yolov3_training_log_resnet18.csv
%env EPOCH=050

## 9. Evaluate retrained model <a class="anchor" id="head-9"></a>

In [ ]:
!tao yolo_v3 evaluate -e $SPECS_DIR/yolo_v3_retrain_resnet18_tfrecord.txt \
                      -m $EXPERIMENT_DIR/experiment_dir_retrain/weights/yolov3_resnet18_epoch_$EPOCH.tlt \
                      -k $KEY

## 10. Visualize inferences <a class="anchor" id="head-10"></a>
In this section, we run the `infer` tool to generate inferences on the trained models and visualize the results.

In [ ]:
# Copy some test images
!mkdir -p $DATA_DIR/test_samples
!cp $DATA_DIR/testing/image_2/000* $DATA_DIR/test_samples/

In [ ]:
# Running inference for detection on n images
!tao yolo_v3 inference -i $DATA_DIR/test_samples \
                       -o $EXPERIMENT_DIR/yolo_infer_images \
                       -e $SPECS_DIR/yolo_v3_retrain_resnet18_tfrecord.txt \
                       -m $EXPERIMENT_DIR/experiment_dir_retrain/weights/yolov3_resnet18_epoch_$EPOCH.tlt \
                       -l $EXPERIMENT_DIR/yolo_infer_labels \
                       -k $KEY

The `inference` tool produces two outputs. 
1. Overlain images in `$EXPERIMENT_DIR/yolo_infer_images`
2. Frame by frame bbox labels in kitti format located in `$EXPERIMENT_DIR/yolo_infer_labels`

In [ ]:
# Simple grid visualizer
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
!mkdir -p $EXPERIMENT_DIR/yolo_infer_images
# Visualizing the sample images.
OUTPUT_PATH = 'yolo_infer_images' # relative path from $EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)